In [ ]:
import gradio as gr
import numpy as np
import os
from model import main, load_trained_model, preprocess_input, predict
from transformers import pipeline

# Create Model
if not os.path.exists('diabetes_model.h5'):
    main()
else:
    print('Model already exists')
# Load your trained model
model = load_trained_model('diabetes_model.h5')

# Prediction function
def predict_diabetes(HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack,
                     PhysActivity, Fruits, Veggies, HvyAlcoholConsump, GenHlth,
                     MentHlth, PhysHlth, DiffWalk, Sex, Age, Education, Income,
                     AnyHealthcare, NoDocbcCost):

    # Collect user input data into a list
    user_data = [HighBP, HighChol, CholCheck, BMI, Smoker, Stroke, HeartDiseaseorAttack,
                     PhysActivity, Fruits, Veggies, HvyAlcoholConsump, GenHlth,
                     MentHlth, PhysHlth, DiffWalk, Sex, Age, Education, Income,
                     AnyHealthcare, NoDocbcCost]
    
    # Replace None with 0
    user_data = [0 if v is None else v for v in user_data]

    # Print user_data to inspect the values
    print(f"User data: {user_data}")

    # Convert to a 2D numpy array (1 sample, 20 features)
    user_data_np = np.array(user_data)  

    # Flatten the array to ensure shape is (1, 20)
    # user_data_np = user_data_np.reshape(1, -1)

    # Ensure the shape is correct before preprocessing
    print(f"User data shape after reshaping: {user_data_np.shape}")

    # Preprocess the input if needed
    processed_data = preprocess_input(user_data_np)
    
    # Ensure the processed data is reshaped correctly before prediction
    # processed_data = np.array(processed_data).reshape(1, -1)  # Reshape to (1, 20) if needed

    # Predict using the loaded model
    prediction = predict(processed_data, model)

    return prediction

# Conversation model for chatbot
class ConversationModel:
    def __init__(self, model_name="m42-health/Llama3-Med42-8B"):
        self.pipe = pipeline("text-generation", model=model_name)

    def ask(self, messages, max_new_tokens=200):
        response = self.pipe(messages, max_length=1024, do_sample=True)
        return response[0]['generated_text']

# Initialize the chatbot
chatbot_model = ConversationModel()

# Function to handle chatbot conversation
def chat_with_bot(user_message):
    return chatbot_model.ask(user_message)

# Define the Gradio interface
diabetes_interface = gr.Interface(
    fn=predict_diabetes,
    inputs=[
        gr.components.Dropdown([0, 1], label="High Blood Pressure (HighBP)"),
        gr.components.Dropdown([0, 1], label="Cholesterol Check (CholCheck)"),
        gr.components.Dropdown([0, 1], label="High Cholesterol"),
        gr.components.Slider(10, 50, step=1, label="BMI"),
        gr.components.Dropdown([0, 1], label="Smoker"),
        gr.components.Dropdown([0, 1], label="Stroke"),
        gr.components.Dropdown([0, 1], label="Heart Disease or Attack"),
        gr.components.Dropdown([0, 1], label="Physical Activity (PhysActivity)"),
        gr.components.Dropdown([0, 1], label="Eats Fruits"),
        gr.components.Dropdown([0, 1], label="Eats Veggies"),
        gr.components.Dropdown([0, 1], label="Heavy Alcohol Consumption"),
        gr.components.Slider(1, 5, step=1, label="General Health (GenHlth)"),
        gr.components.Slider(0, 30, step=1, label="Mental Health Days (MentHlth)"),
        gr.components.Slider(0, 30, step=1, label="Physical Health Days (PhysHlth)"),
        gr.components.Dropdown([0, 1], label="Difficulty Walking (DiffWalk)"),
        gr.components.Dropdown([0, 1], label="Sex (0=Female, 1=Male)"),
        gr.components.Slider(18, 80, step=1, label="Age"),
        gr.components.Slider(1, 12, step=1, label="Education Level"),
        gr.components.Slider(1, 8, step=1, label="Income Level"),
        gr.components.Dropdown([0, 1], label="Any Healthcare Access"),
        gr.components.Dropdown([0, 1], label="No Doctor Because of Cost"),
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="Diabetes Assesment Tool",
    description="This interface is a risk assesment tool and not a diagnostic. If results are positive we recommend visiting your physician to confirm your risk of diabetes. Enter your information to predict the risk of diabetes (0=no 1=yes). THIS IS A RISK ASSESMENT TOOL NOT A DIAGNOSTIC TOOL. This tool was made using data from the ",
)
# Chatbot interface for medical conversations
chat_interface = gr.Interface(
    fn=chat_with_bot,
    inputs=gr.Textbox(lines=2, label="Ask a medical question"),
    outputs=gr.Textbox(label="Chatbot Response"),
    title="Medical Chatbot",
    description="Ask any medical-related question, and the bot will respond."
)

# Create a Tabbed Interface combining both
tabbed_interface = gr.TabbedInterface([diabetes_interface, chat_interface], ["Diabetes Prediction", "Medical Chatbot"])


# Launch the combined interface
tabbed_interface.launch()
